In [74]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [129]:
import pandas as pd
import numpy as np

data4 = pd.read_csv("/content/drive/MyDrive/КНИР Васильев Игорь БПМ19-4/Разные модели/sqsq.csv")
data4 = data4.drop(data4[data4.Label == 'Label'].index)

NORM_LEN = 2000
DDOS_LEN = 2000
BRUTE_LEN = 1500
SQL_LEN = 2500
data1 = pd.read_csv("/content/drive/MyDrive/КНИР Васильев Игорь БПМ19-4/Разные модели/norm.csv")
data2 = pd.read_csv("/content/drive/MyDrive/КНИР Васильев Игорь БПМ19-4/Разные модели/ddos.csv")
data3 = pd.read_csv("/content/drive/MyDrive/КНИР Васильев Игорь БПМ19-4/Разные модели/nb1.csv")
data1 = data1.sample(NORM_LEN)
data2 = data2.sample(DDOS_LEN)
data3 = data3.sample(BRUTE_LEN)
df = data4.sample(SQL_LEN)
df = df.loc[df['Label'] == 'Infilteration']
df2 = data4.sample(1000)
data = pd.concat([data1, data2, data3, df, df2], ignore_index=True, sort=False)
shuffle = data.sample(frac=1, random_state=101).reset_index()

<ipython-input-129-96cf5a3fa1ae>:4: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78) have mixed types. Specify dtype option on import or set low_memory=False.
  data4 = pd.read_csv("/content/drive/MyDrive/КНИР Васильев Игорь БПМ19-4/Разные модели/sqsq.csv")


In [130]:
shuffle.fillna(0, inplace=True)
df = shuffle

In [131]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [132]:
test_df = df.copy()

In [133]:
test_df.replace([np.inf, -np.inf], np.nan, inplace=True)
test_df.dropna(how="all", inplace=True)

number_inf = test_df[test_df == np.inf].count()

In [134]:
def ip2int(ip):
    if ip == 0:
      ip = '0.0.0.0'
    ip_list = list(map(int, ip.split('.')))
    num = 0
    for i in ip_list:
        num *= 256
        num += i
    return num

In [135]:
def delete_first_zero(val):
    if val[0] == '0':
        return val[1:]
    return val

def time2int(tm):
    tm_split = tm.split(' ')
    day_info = tm_split[0].split('/')
    day_info = list(map(delete_first_zero, day_info))
    day_info = list(map(int, day_info))
    day_amount = day_info[0] + (day_info[1] - 1) * 30 + (day_info[2] - 1970) * 365
    
    time_info = tm_split[1].split(':')
    time_info = list(map(delete_first_zero, time_info))
    time_info = list(map(int, time_info))
    sec_amount = 60 * (time_info[0] * 60 + time_info[1]) + time_info[2]
    
    if len(tm_split) == 3 and tm_split[-1] == 'PM':
        sec_amount += 3600 * 12
    sec_amount += day_amount * 24 * 3600
    
    return sec_amount

In [136]:
y = test_df['Label']
test_df = test_df.drop(['Label'], axis=1)
test_df['Src IP'] = test_df['Src IP'].apply(ip2int)
test_df['Dst IP'] = test_df['Dst IP'].apply(ip2int)
test_df['Timestamp'] = test_df['Timestamp'].apply(time2int)

In [137]:
test_df = test_df.drop(['Flow ID'], axis=1)

In [84]:
IMPORTANT_COLS = ['Src IP',
                  'Dst IP',
                  'Timestamp',
                  'Src Port',
                  'Dst Port',
                  'Protocol','Flow Duration',
                  'Flow Byts/s',
                  'Flow Pkts/s',
                  'Pkt Len Var',
                  'Down/Up Ratio'
                 ]

In [85]:
test_df = test_df.drop([i for i in list(test_df.columns) if i not in IMPORTANT_COLS], axis=1)
test_df.head()

,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Flow Byts/s,Flow Pkts/s,Pkt Len Var,Down/Up Ratio
0,3232249857,37606.0,1632266441,80,6,1275509170,63056826,13.384752,0.301316,13840.589474,0.0
1,0,0.0,0,21,6,1517570217,3,0.000000,666666.666667,0.000000,1.0
2,2887729983,53605.0,2887712770,53,17,1518087757,1839,121805.328983,1087.547580,6533.333333,1.0
3,3232249857,37740.0,1632276588,80,6,1275567973,83569276,8.986556,0.215390,14584.374269,0.0
4,316396426,36016.0,2887730457,80,6,1517684921,11804738,112.243067,0.677694,109290.444444,1.0


In [138]:
test_df = test_df.fillna(0)

In [139]:
test_df = test_df.astype(np.float)

<ipython-input-139-761d90559372>:1: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  test_df = test_df.astype(np.float)


In [140]:
df_z_scaled = test_df.copy()
  
# apply normalization techniques
for column in df_z_scaled.columns:
    df_z_scaled[column] = (df_z_scaled[column] -
                           df_z_scaled[column].mean()) / df_z_scaled[column].std()
df_z_scaled = df_z_scaled.fillna(0)
df_z_scaled.head()

,index,Unnamed: 0,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,Bwd Pkt Len Min,Bwd Pkt Len Mean,Bwd Pkt Len Std,Flow Byts/s,Flow Pkts/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Tot,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Tot,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Len,Bwd Header Len,Fwd Pkts/s,Bwd Pkts/s,Pkt Len Min,Pkt Len Max,Pkt Len Mean,Pkt Len Std,Pkt Len Var,FIN Flag Cnt,SYN Flag Cnt,RST Flag Cnt,PSH Flag Cnt,ACK Flag Cnt,URG Flag Cnt,CWE Flag Count,ECE Flag Cnt,Down/Up Ratio,Pkt Size Avg,Fwd Seg Size Avg,Bwd Seg Size Avg,Fwd Byts/b Avg,Fwd Pkts/b Avg,Fwd Blk Rate Avg,Bwd Byts/b Avg,Bwd Pkts/b Avg,Bwd Blk Rate Avg,Subflow Fwd Pkts,Subflow Fwd Byts,Subflow Bwd Pkts,Subflow Bwd Byts,Init Fwd Win Byts,Init Bwd Win Byts,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,0.474419,-0.527865,-0.985678,-0.779928,-0.974738,-0.326184,-0.437911,0.552442,-0.398352,-0.057354,-0.041361,-0.238618,-0.030849,-0.536119,-0.308459,-0.603299,-0.517018,-0.567093,-0.355836,-0.506673,-0.525052,0.0,0.0,-0.232757,-0.296323,-0.364958,-0.158066,-0.350204,-0.231985,-0.281942,-0.316256,-0.129593,-0.308905,-0.204253,-0.268422,-0.285959,-0.084946,-0.171487,-0.070834,-0.023559,0.0,-0.050902,-0.044953,1.088864,1.217716,-0.340315,-0.589019,-0.552846,-0.577454,-0.352758,-0.178226,-0.513583,-0.319787,1.178476,-0.527785,-0.152076,-0.023559,-0.318996,0.281875,-0.585777,-0.603299,-0.506673,0.0,0.0,0.0,0.0,0.0,0.0,-0.057354,-0.238618,-0.041361,-0.030849,1.545759,-0.437614,-0.395203,1.599472,-0.126853,-0.077559,-0.130435,-0.114065,-0.307987,-0.168301,-0.320767,-0.288947
1,-0.253424,-0.427161,1.145248,0.706622,-0.152351,-0.321957,-0.437911,-1.805205,-0.398352,-0.069582,-0.035030,-0.238618,-0.030849,-0.536119,-0.308459,-0.603299,-0.517018,-0.567093,-0.355836,-0.506673,-0.525052,0.0,0.0,-0.232757,-0.296323,-0.364958,-0.158065,-0.350204,-0.231985,-0.281942,-0.316256,-0.129593,-0.308904,-0.204252,-0.268422,-0.285958,-0.084944,-0.171487,-0.070834,-0.023559,0.0,-0.075199,-0.031028,-0.468815,0.363447,-0.340315,-0.589019,-0.552846,-0.577454,-0.352758,5.610090,-0.513583,-0.319787,-0.848436,1.894449,-0.152076,-0.023559,-0.318996,-1.181277,-0.585777,-0.603299,-0.506673,0.0,0.0,0.0,0.0,0.0,0.0,-0.069582,-0.238618,-0.035030,-0.030849,-0.642558,-0.425654,-0.395203,-1.100833,-0.126853,-0.077559,-0.130435,-0.114065,-0.307987,-0.168301,-0.320767,-0.288947
2,-1.552012,0.524026,0.918116,1.692432,1.111095,-0.323891,2.150845,0.557273,-0.398341,-0.057354,-0.041361,-0.200875,-0.030582,-0.352165,1.350261,0.093171,-0.517018,-0.466438,0.783199,-0.245845,-0.525052,0.0,0.0,-0.232728,-0.296323,-0.364939,-0.158036,-0.350204,-0.231985,-0.281942,-0.316256,-0.129593,-0.308905,-0.204253,-0.268422,-0.285959,-0.084946,-0.171487,-0.070834,-0.023559,0.0,-0.070340,-0.048751,-0.458326,-0.479316,1.500397,-0.488809,-0.211030,-0.531194,-0.352191,-0.178226,-0.513583,-0.319787,-0.848436,-0.527785,-0.152076,-0.023559,-0.318996,0.281875,-0.104984,0.093171,-0.245845,0.0,0.0,0.0,0.0,0.0,0.0,-0.057354,-0.200875,-0.041361,-0.030582,-0.642558,-0.437663,-0.395203,-0.560772,-0.126853,-0.077559,-0.130435,-0.114065,-0.307987,-0.168301,-0.320767,-0.288947
3,-0.380256,-0.487210,1.145239,-0.691176,1.488758,-0.321957,-0.437911,-1.805155,-0.398338,-0.057354,-0.041361,-0.238618,-0.030849,-0.536119,-0.308459,-0.603299,-0.517018,-0.567093,-0.355836,-0.506673,-0.525052,0.0,0.0,-0.232719,-0.296323,-0.364933,-0.158026,-0.350204,-0.231985,-0.281942,-0.316256,-0.129593,-0.308905,-0.204253,-0.268422,-0.285959,-0.084946,-0.171487,-0.070834,-0.023559,0.0,-0.063051,-0.044953,-0.460827,-0.482060,-0.340315,-0.589019,-0.552846,-0.577454,-0.352758,-0.178226,-0.513583,-0.319787,-0.848436,1.894449,-0.152076

In [141]:
y = y.str.replace('ddos', '1').str.replace('Benign','0').str.replace('FTP-BruteForce','2').str.replace('Infilteration','3')
y = y.astype(np.int64)

In [142]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_z_scaled, y, test_size=0.3, random_state=42)

In [143]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()

parameters = {"n_neighbors": range(1, 30)}
gridsearch = GridSearchCV(knn, parameters,return_train_score=True, refit=True)
gridsearch.fit(X_train, y_train)

GridSearchCV(estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': range(1, 30)}, return_train_score=True)

In [144]:
gridsearch.best_params_

{'n_neighbors': 4}

In [145]:
y_pred = gridsearch.predict(X_test)

In [146]:
from sklearn.metrics import classification_report, accuracy_score
print('Results on the test set:')
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test,y_pred))

Results on the test set:
              precision    recall  f1-score   support

           0       0.90      0.94      0.92       825
           1       0.99      0.99      0.99       591
           2       1.00      1.00      1.00       460
           3       0.81      0.69      0.75       287

    accuracy                           0.93      2163
   macro avg       0.92      0.91      0.91      2163
weighted avg       0.93      0.93      0.93      2163

0.9348127600554785
